In [2]:
import os
import torch
import torchvision
import numpy as np
import cv2
from tqdm import tqdm
from torchvision.transforms.functional import to_tensor
from torch.utils.data import DataLoader, Dataset
from torchvision.models.detection import maskrcnn_resnet50_fpn
import matplotlib.pyplot as plt
import random

In [3]:
class CellDataset(Dataset):
    def __init__(self, image_dir, mask_dir):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.images = sorted(os.listdir(image_dir))
        self.masks = sorted(os.listdir(mask_dir))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.masks[idx])

        # 이미지 로드 및 검증
        img = cv2.imread(img_path)
        if img is None:  # 이미지가 None일 경우 건너뛰기
            print(f"[ERROR] Cannot read image: {img_path}")
            return None

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # 마스크 로드 및 검증
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:  # 마스크가 None일 경우 건너뛰기
            print(f"[ERROR] Cannot read mask: {mask_path}")
            return None

        # Tensor로 변환
        img = to_tensor(img).clamp(0, 1)
        mask = torch.as_tensor(mask, dtype=torch.uint8)

        obj_ids = torch.unique(mask)[1:]  # 배경(0) 제외
        masks = mask == obj_ids[:, None, None]

        boxes = []
        for i in range(len(obj_ids)):
            pos = torch.where(masks[i])
            xmin, xmax = torch.min(pos[1]), torch.max(pos[1])
            ymin, ymax = torch.min(pos[0]), torch.max(pos[0])
            boxes.append([xmin.item(), ymin.item(), xmax.item(), ymax.item()])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones((len(obj_ids),), dtype=torch.int64)

        target = {"boxes": boxes, "labels": labels, "masks": masks}

        return img, target


In [4]:
# Model Initialization
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = maskrcnn_resnet50_fpn(weights="DEFAULT")

# Redefine Predictors
num_classes = 2
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
hidden_layer = 256
model.roi_heads.mask_predictor = torchvision.models.detection.mask_rcnn.MaskRCNNPredictor(
    in_features_mask, hidden_layer, num_classes
)

# Set image size
model.to(device)

# Paths to datasets
train_image_dir = r"C:\Users\user\Desktop\Cell DIC Mask R-CNN\png_convert\Training_data_512\train\DIC"
train_mask_dir = r"C:\Users\user\Desktop\Cell DIC Mask R-CNN\png_convert\Training_data_512\train\mask"
val_image_dir = r"C:\Users\user\Desktop\Cell DIC Mask R-CNN\png_convert\Training_data_512\val\DIC"
val_mask_dir = r"C:\Users\user\Desktop\Cell DIC Mask R-CNN\png_convert\Training_data_512\val\mask"

# Create datasets
train_dataset = CellDataset(image_dir=train_image_dir, mask_dir=train_mask_dir)
val_dataset = CellDataset(image_dir=val_image_dir, mask_dir=val_mask_dir)

print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")




Training dataset size: 7014
Validation dataset size: 1986


In [5]:
def validate_files(image_dir, mask_dir):
    image_files = sorted(os.listdir(image_dir))
    mask_files = sorted(os.listdir(mask_dir))

    for img_file, mask_file in zip(image_files, mask_files):
        img_path = os.path.join(image_dir, img_file)
        mask_path = os.path.join(mask_dir, mask_file)

        if cv2.imread(img_path) is None:
            print(f"[ERROR] Invalid image file: {img_path}")
        if cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) is None:
            print(f"[ERROR] Invalid mask file: {mask_path}")

dic_dir = r"C:\Users\user\Desktop\Cell DIC Mask R-CNN\png_convert\Training_data_512\train\DIC"
mask_dir = r"C:\Users\user\Desktop\Cell DIC Mask R-CNN\png_convert\Training_data_512\train\mask"

dic_dir = r"C:\Users\user\Desktop\Cell DIC Mask R-CNN\png_convert\Training_data_512\val\DIC"
mask_dir = r"C:\Users\user\Desktop\Cell DIC Mask R-CNN\png_convert\Training_data_512\val\mask"

# 파일 유효성 검사
validate_files(dic_dir, mask_dir)

In [6]:
import os
import random
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm

# Model Transform Settings for 512x512
#model.transform.min_size = [512]
#model.transform.max_size = 512

# Adjust Anchor Settings
#model.rpn.anchor_generator.sizes = ((32, 64, 128),)  # Smaller anchors for smaller patches
#model.rpn.anchor_generator.aspect_ratios = ((0.5, 1.0, 2.0),)  # Default aspect ratios

# DataLoader
def collate_fn(batch):
    batch = [b for b in batch if b is not None]  # None 데이터 제거
    if len(batch) == 0:  # 모든 데이터가 None이면 None 반환
        return None
    return tuple(zip(*batch))

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)

# Optimizer and Scheduler
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Training Loop
num_epochs = 30
log_dir = "training_logs_512"
os.makedirs(log_dir, exist_ok=True)

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    print(f"\nEpoch {epoch + 1}/{num_epochs}")

    # Training Phase
    for step, batch in enumerate(tqdm(train_loader, desc="Training_512")):
        if batch is None:  # 빈 배치 건너뛰기
            print(f"Step {step}: Empty batch detected, skipping...")
            continue

        imgs, targets = batch
        imgs = [img.to(device) for img in imgs]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Forward pass와 Loss 계산
        loss_dict = model(imgs, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        epoch_loss += losses.item()

        # Display Loss every 300 steps
        if step % 300 == 0:
            print(f"Step [{step}/{len(train_loader)}], Loss: {losses.item():.4f}")

        # Save Validation Visualization every 500 steps
        if step % 500 == 0 and step != 0:
            model.eval()
            with torch.no_grad():
                random_idx = random.randint(0, len(val_dataset) - 1)
                val_img, val_target = val_dataset[random_idx]
                val_img = val_img.to(device).unsqueeze(0)
                prediction = model(val_img)

                # Visualization
                plt.figure(figsize=(12, 6))
                plt.subplot(1, 2, 1)
                plt.title("Input Image")
                plt.imshow(val_img[0].cpu().permute(1, 2, 0))
                plt.axis("off")

                plt.subplot(1, 2, 2)
                plt.title("Predicted Mask")
                if "masks" in prediction[0] and prediction[0]["masks"].shape[0] > 0:
                    pred_mask = prediction[0]["masks"].cpu().numpy()
                    combined_mask = pred_mask.max(axis=0).squeeze()
                    plt.imshow(combined_mask, cmap="gray")
                else:
                    plt.text(0.5, 0.5, "No mask predicted", ha="center", va="center", fontsize=12)
                plt.axis("off")

                # Save Visualization
                plt_path = os.path.join(log_dir, f"epoch_{epoch + 1}_step_{step}.png")
                plt.savefig(plt_path)
                print(f"Saved visualization: {plt_path}")
                plt.close()
            model.train()

    print(f"Epoch {epoch + 1} Loss: {epoch_loss:.4f}")
    lr_scheduler.step()

    # Save Model Checkpoint
    if (epoch + 1) % 5 == 0:
        checkpoint_path = os.path.join(log_dir, f"mask_rcnn_epoch_{epoch + 1}.pth")
        torch.save(model.state_dict(), checkpoint_path)
        print(f"Checkpoint saved: {checkpoint_path}")

# Save Final Model
final_model_path = os.path.join(log_dir, "mask_rcnn_final.pth")
torch.save(model.state_dict(), final_model_path)
print(f"Final model saved: {final_model_path}")



Epoch 1/30


Training_512:   0%|                                                                                         | 0/877 [00:00<?, ?it/s]C:\Users\user\anaconda3\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
C:\Users\user\anaconda3\Lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Training_512:   0%|                                                                                

Step [0/877], Loss: 3.4895


Training_512:  34%|███████████████████████████                                                    | 301/877 [03:39<07:01,  1.37it/s]

Step [300/877], Loss: 0.2775


...

Saved visualization: training_logs_512\epoch_30_step_500.png


Training_512:  69%|▋| 601/877 [06:57<03:11,  1.

Step [600/877], Loss: 0.1137


Training_512: 100%|█| 877/877 [10:08<00:00,  1.


Epoch 30 Loss: 117.5089
Checkpoint saved: training_logs_512\mask_rcnn_epoch_30.pth
Final model saved: training_logs_512\mask_rcnn_final.pth
